In [1]:
from models.model import Model
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer
import torch

In [4]:
df_book = pd.read_csv("/home/macab/research/Adaptil/data/amazon-review/books.csv")
df_book_ul = pd.read_csv("/home/macab/research/Adaptil/data/amazon-review/books-UL.csv")


In [9]:
df_book.tail()

,product_type,helpful,rating,title,review_text,sentiment
1982,books,NaN,4.0,Check the facts,Girl Sleuth is a must-read especially for the ...,1
1983,books,NaN,2.0,Why the music?,I find some parts of Didion's book very relata...,0
1984,books,27 of 33,1.0,NOT FOR A FAMILY WITH KIDS,"The photos are beautiful, the designs are just...",0
1985,books,NaN,5.0,"A Fantastic Exercise in ""What If""",This isn't really a novel and it isn't really ...,1
1986,books,3 of 8,1.0,A major disservice to the book,The book THE LOST CONTINENT is an amazing capt...,0


In [6]:
df_book_ul.head()

,product_type,helpful,rating,title,review_text
0,books,1 of 1,4.0,Insightful and thoroughly researched from my p...,I'm an employee with a small educational busin...
1,books,3 of 13,2.0,Melodramatic,McCarthy is a very talented writer (read Blood...
2,books,NaN,5.0,The Dog Chapel,"Not only is the story heartwarming, the artwor..."
3,books,1 of 1,5.0,Another one for my collection!,"Kathy's books are my favorites, and I will alw..."
4,books,NaN,1.0,The Insider's Guide to Making Money in Real Es...,This Dolf de Roos book is no different from hi...


In [7]:
df_book.shape

(1987, 6)

In [8]:
df_book_ul.shape

(5000, 5)

In [4]:
data = load_dataset("amazon_reviews_multi", name="en")

Downloading: 100%|██████████| 2.06M/2.06M [00:06<00:00, 337kB/s]
Downloading: 100%|██████████| 2.05M/2.05M [00:08<00:00, 243kB/s]
Dataset amazon_reviews_multi downloaded and prepared to /home/macab/.cache/huggingface/datasets/amazon_reviews_multi/en/1.0.0/f3357bd271e187385a38574fe31b8fb10055303f67fa9fce55e84d08c4870efd. Subsequent calls will reuse this data.


In [9]:
# data['train']['product_category']

In [41]:
DOMAIN = "kitchen"

In [42]:
book = data.filter(lambda example: example['product_category']==DOMAIN)

100%|██████████| 200/200 [00:06<00:00, 33.04ba/s]
Loading cached processed dataset at /home/macab/.cache/huggingface/datasets/amazon_reviews_multi/en/1.0.0/f3357bd271e187385a38574fe31b8fb10055303f67fa9fce55e84d08c4870efd/cache-9d2f6f431303cdfa.arrow
Loading cached processed dataset at /home/macab/.cache/huggingface/datasets/amazon_reviews_multi/en/1.0.0/f3357bd271e187385a38574fe31b8fb10055303f67fa9fce55e84d08c4870efd/cache-308b7c2cdd6eda2a.arrow


In [45]:
len(book['train'])

10382

In [57]:
count = 0

for each in data['train']['product_category']:
    if each == "movie":
        count += 1

print(count)

0


In [ ]:
dataset = load_dataset("multi_nli")
# DOMAIN in ['government', 'telephone', 'fiction', 'travel', 'slate'] to train
# which user will enter
domain_dataset = dataset.filter(lambda example: example['genre'].startswith(DOMAIN))
domain_dataset['train'] = domain_dataset['train'].select(range(77306))  # same for training across all domains
domain_dataset['validation_matched'] = domain_dataset['validation_matched'].select(range(1945)) # same for validation across all domains
encoded_domain_dataset = domain_dataset.map(lambda x: tokenizer(x['premise'], x['hypothesis'], padding='max_length', truncation=True), batched=True)
encoded_domain_dataset['train'].set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
encoded_domain_dataset['validation_matched'].set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
train_data_loader = torch.utils.data.DataLoader(dataset = encoded_domain_dataset['train'], batch_size=32, shuffle=True, num_workers=4)
val_data_loader = torch.utils.data.DataLoader(dataset = encoded_domain_dataset['validation_matched'], batch_size=32, shuffle=False, num_workers=4)

In [15]:
data = load_dataset("csv", data_files="/home/macab/research/Adaptil/data/amazon-review/books.csv")

Using custom data configuration default


0 tables [00:00, ? tables/s]

1 tables [00:00,  7.98 tables/s]

                                Dataset csv downloaded and prepared to /home/macab/.cache/huggingface/datasets/csv/default-3c962ed2152802d3/0.0.0/49187751790fa4d820300fd4d0707896e5b941f1a9c644652645b866716a4ac4. Subsequent calls will reuse this data.


In [21]:
len(data['train']['review_text'])

1987

In [22]:
data['train']['review_text'][0]

'This is wonderful! It is for fans of Laura Ingalls Wilder, and also for those who are interested in historic cooking - from prairie and colonial days which have their basis in so many of the cultures which settled America in the early days, and adapted for use there'

In [23]:
len(data['train']['review_text'])

1987

In [32]:
train = load_dataset("csv", data_files="/home/macab/research/Adaptil/data/amazon-review/books.csv", split="train[:90%]")
valid = load_dataset("csv", data_files="/home/macab/research/Adaptil/data/amazon-review/books.csv", split="train[10%:]")

Using custom data configuration default
Reusing dataset csv (/home/macab/.cache/huggingface/datasets/csv/default-3c962ed2152802d3/0.0.0/49187751790fa4d820300fd4d0707896e5b941f1a9c644652645b866716a4ac4)
Using custom data configuration default
Reusing dataset csv (/home/macab/.cache/huggingface/datasets/csv/default-3c962ed2152802d3/0.0.0/49187751790fa4d820300fd4d0707896e5b941f1a9c644652645b866716a4ac4)


In [34]:
len(train['review_text'])

1788

In [44]:
# train['sentiment']

In [60]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased', usefast=True, use_lower_case=True)

In [38]:
encoded_dataset = train.map(lambda x: tokenizer(x['review_text'], padding='max_length', truncation=True), batched=True)



  0%|          | 0/2 [00:00<?, ?ba/s]

 50%|█████     | 1/2 [00:02<00:02,  2.62s/ba]

100%|██████████| 2/2 [00:04<00:00,  2.17s/ba]


In [48]:
torch.tensor([encoded_dataset[0]['sentiment']])

tensor([1])

In [49]:
encoded_dataset = encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'sentiment'])

In [62]:
# load_dataset?

In [2]:
from dataset.dataset import create_sa_loaders
from transformers import AutoTokenizer
from datasets import load_dataset
from tqdm import tqdm


ImportError: cannot import name 'create_sa_loaders' from 'dataset.dataset' (/home/macab/research/Adaptil/dataset/dataset.py)

In [2]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased', usefast=True, use_lower_case=True)

In [3]:
loaders = create_sa_loaders(tokenizer=tokenizer, root_dir="/home/macab/research/Adaptil/data/amazon-review/")

  0%|          | 0/4 [00:00<?, ?it/s]Using custom data configuration default
Reusing dataset csv (/home/macab/.cache/huggingface/datasets/csv/default-3c962ed2152802d3/0.0.0/49187751790fa4d820300fd4d0707896e5b941f1a9c644652645b866716a4ac4)
 25%|██▌       | 1/4 [00:01<00:05,  1.93s/it]Using custom data configuration default
Reusing dataset csv (/home/macab/.cache/huggingface/datasets/csv/default-b4d4281f510cdb95/0.0.0/49187751790fa4d820300fd4d0707896e5b941f1a9c644652645b866716a4ac4)
 50%|█████     | 2/4 [00:03<00:03,  1.90s/it]Using custom data configuration default
Reusing dataset csv (/home/macab/.cache/huggingface/datasets/csv/default-9c3cc848a6ee4edd/0.0.0/49187751790fa4d820300fd4d0707896e5b941f1a9c644652645b866716a4ac4)
 75%|███████▌  | 3/4 [00:04<00:01,  1.68s/it]Using custom data configuration default
Reusing dataset csv (/home/macab/.cache/huggingface/datasets/csv/default-ffd15131893ced8b/0.0.0/49187751790fa4d820300fd4d0707896e5b941f1a9c644652645b866716a4ac4)
100%|██████████| 4/4

In [5]:
len(loaders['books']['train'])

187

In [7]:
batch_1 = next(iter(loaders['books']['train']))
batch_2 = next(iter(loaders['books']['valid']))

In [8]:
batch_1['label']

tensor([[1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.]])

In [9]:
batch_2['label']

tensor([[0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.]])

In [6]:
batch = next(iter(loaders['books']['train']))

In [7]:
# batch['input_ids'].shape

torch.Size([8, 128])

In [4]:
data = load_dataset("csv", data_files="/home/macab/research/Adaptil/data/amazon-review/books.csv")

Using custom data configuration default
Reusing dataset csv (/home/macab/.cache/huggingface/datasets/csv/default-3c962ed2152802d3/0.0.0/49187751790fa4d820300fd4d0707896e5b941f1a9c644652645b866716a4ac4)


In [12]:
root_dir = "/home/macab/research/Adaptil/data/amazon-review/"

In [5]:
loaders = {}

In [9]:
domains =["books", "dvd", "electronics", "kitchen_housewares"]

In [15]:
for domain in tqdm(domains):
    
    d = os.path.join(root_dir, domain+".csv")
    
    # load the data
    train = load_dataset("csv", data_files=d)
    # valid = load_dataset("csv", data_files=d, split="train[75%:]")
    
    
    # map it with tokenizer 
    train_dataset = train['train'].map(lambda x: tokenizer(x['review_text'], padding='max_length', truncation=True), batched=True)
    # valid_dataset = valid.map(lambda x: tokenizer(x['review_text'], padding='max_length', truncation=True), batched=True)
    
    
    # convert into tensors 
    train_dataset = train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'sentiment'])
    # valid_dataset = valid_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'sentiment'])
    
    # create loader
    train_loader = data.DataLoader(dataset = train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    # valid_loader = data.DataLoader(dataset = valid_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    
    
    # put the domain laoder into dictionary
    loaders[domain] = {
        "train":train_loader,
        # "valid":valid_loader
    }




  0%|          | 0/4 [00:00<?, ?it/s]Using custom data configuration default
Reusing dataset csv (/home/macab/.cache/huggingface/datasets/csv/default-3c962ed2152802d3/0.0.0/49187751790fa4d820300fd4d0707896e5b941f1a9c644652645b866716a4ac4)

  0%|          | 0/4 [00:06<?, ?it/s]


NameError: name 'data' is not defined

In [19]:
 # load the data
train = load_dataset("csv", data_files=d)
# valid = load_dataset("csv", data_files=d, split="train[75%:]")


# map it with tokenizer 
train_dataset = train.map(lambda x: tokenizer(x['review_text'], padding='max_length', truncation=True), batched=True)
# valid_dataset = valid.map(lambda x: tokenizer(x['review_text'], padding='max_length', truncation=True), batched=True)


# convert into tensors 
train_dataset = train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'sentiment'])
# valid_dataset = valid_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'sentiment'])

Using custom data configuration default
Reusing dataset csv (/home/macab/.cache/huggingface/datasets/csv/default-3c962ed2152802d3/0.0.0/49187751790fa4d820300fd4d0707896e5b941f1a9c644652645b866716a4ac4)
Loading cached processed dataset at /home/macab/.cache/huggingface/datasets/csv/default-3c962ed2152802d3/0.0.0/49187751790fa4d820300fd4d0707896e5b941f1a9c644652645b866716a4ac4/cache-4072fc88f9705a80.arrow


In [20]:
type(train_dataset)

NoneType

In [21]:
train

DatasetDict({'train': Dataset(features: {'product_type': Value(dtype='string', id=None), 'helpful': Value(dtype='string', id=None), 'rating': Value(dtype='float64', id=None), 'title': Value(dtype='string', id=None), 'review_text': Value(dtype='string', id=None), 'sentiment': Value(dtype='int64', id=None)}, num_rows: 1987)})

### Testing the Pipelinne

In [3]:
from transformers import AutoModel, AutoConfig, AutoTokenizer 
from dataset.dataset import sa_loaders


In [5]:
models = ['bert-base-uncased', 'distilbert-base-uncased', 'roberta-base', 'distilroberta-base']

In [3]:
# for model in models:

#     config = AutoConfig.from_pretrained(model)

#     print(config.hidden_size)
#     print(config.dropout)

In [4]:
# config = AutoConfig.from_pretrained('distilbert-for-sequence-classification')

In [5]:
from datasets import load_dataset
train_dataset = load_dataset('csv', data_files="/home/macab/research/Adaptil/data/amazon-review/books.csv", split='train[:1582]')
        
train_dataset = train_dataset.rename_column('sentiment', 'label') 


Using custom data configuration default-9729425060717359
Reusing dataset csv (/home/macab/.cache/huggingface/datasets/csv/default-9729425060717359/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0)


In [6]:
train_dataset

Dataset({
    features: ['product_type', 'helpful', 'rating', 'title', 'review_text', 'label'],
    num_rows: 1582
})

In [6]:
tokenizer = AutoTokenizer.from_pretrained(models[0])

In [7]:
loaders = sa_loaders(tokenizer=tokenizer)

Using custom data configuration default-9729425060717359
Reusing dataset csv (/home/macab/.cache/huggingface/datasets/csv/default-9729425060717359/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0)
Using custom data configuration default-9729425060717359
Reusing dataset csv (/home/macab/.cache/huggingface/datasets/csv/default-9729425060717359/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0)
100%|██████████| 1/1 [00:00<00:00, 17.84ba/s]
Using custom data configuration default-4ddecc39120ba9da
Reusing dataset csv (/home/macab/.cache/huggingface/datasets/csv/default-4ddecc39120ba9da/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0)
Using custom data configuration default-4ddecc39120ba9da
Reusing dataset csv (/home/macab/.cache/huggingface/datasets/csv/default-4ddecc39120ba9da/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0)
100%|██████████| 1/1 [00:00<00:00, 19.29ba/s]
Using custom data configuration 

In [14]:
batch = next(iter(loaders['books']['train']))

In [10]:
print(batch['input_ids'].shape,  batch['attention_mask'].shape,  batch['label'].shape)

torch.Size([16, 128]) torch.Size([16, 128]) torch.Size([16])


In [8]:
from models.model import Model

In [9]:
model = Model(model_name=models[0])

In [11]:
logits = model(input_ids=batch['input_ids'],  attention_mask=batch['attention_mask'])

In [12]:
logits.shape

torch.Size([2, 2])

In [6]:
from Trainer import LightningModel
from config import config
import pytorch_lightning as pl


In [7]:
lm = LightningModel(model_name=models[0], task_config=config['tasks']['sa'])

In [8]:
trainer = pl.Trainer(
    max_epochs=1
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [9]:
trainer.fit(
    model=lm,
    train_dataloader=loaders['books']['valid'],
    val_dataloaders=loaders['books']['valid']
)


  | Name  | Type  | Params
--------------------------------
0 | model | Model | 110 M 
Validation sanity check:  50%|█████     | 1/2 [00:00<00:00,  1.75it/s]/home/macab/miniconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Epoch 0:   0%|          | 0/396 [00:00<?, ?it/s] /home/macab/miniconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The validation_epoch_end should not return anything as of 9.1.to log, use self.log(...) or self.write(...) directly in the LightningModule
  warnings.warn(*args, **kwargs)
Epoch 0:  15%|█▍        | 59/396 [01:47<10:12,  1.82s/it, loss=0.633, v_num=3]/home/macab/miniconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: Detected Ke

1

In [10]:
# loaders?

In [11]:
from pytorch_lightning.callbacks import ModelCheckpoint

In [13]:
pl.__version__

'1.0.3'

In [1]:
!python main.py

2021-04-24 20:43:30.700468: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-04-24 20:43:30.700497: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/macab/miniconda3/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0
  0%|                                                     | 0/4 [00:00<?, ?it/s]Using custom data configuration default
Reusing dataset csv (/home/macab/.cache/huggingface/datasets/csv/default-2108d0c2